In [ ]:
# setup paths
import sys
basedir = '/Users/nickm/thesis/InvariantPreservingMLSolvers/traditional_demos/1d_compressible_euler'
readwritedir = '/Users/nickm/thesis/InvariantPreservingMLSolvers/traditional_demos/1d_compressible_euler'

sys.path.append('{}/core'.format(basedir))
sys.path.append('{}/simulate'.format(basedir))
sys.path.append('{}/ml'.format(basedir))

In [ ]:
# import external packages
import jax
import jax.numpy as jnp
import numpy as onp
from jax import config, vmap
config.update("jax_enable_x64", True)
import xarray
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from initialconditions import get_a0, f_init_lax_shock_tube, shock_tube_problem_1, shock_tube_problem_2, shock_tube_problem_3, shock_tube_problem_4, shock_tube_problem_5, shock_tube_problem_6, shock_tube_problem_7, shock_tube_problem_8
from simparams import CoreParams, SimulationParams
from simulations import EulerFVSim
from helper import get_rho, get_u, get_p, get_entropy
from trajectory import get_trajectory_fn, get_inner_fn

In [ ]:
def get_core_params(Lx = 1.0, gamma = 5/3, bc = 'periodic', fluxstr = 'laxfriedrichs'):
    return CoreParams(Lx, gamma, bc, fluxstr)


def get_sim_params(name = "test", cfl_safety=0.3, rk='ssp_rk3'):
    return SimulationParams(name, basedir, readwritedir, cfl_safety, rk)


def plot_a(a, core_params, mins = [0.0 - 2e-2] * 3, maxs= [1.0 + 5e-2] * 3):
    x = jnp.linspace(0.0, core_params.Lx, a.shape[1])
    
    fig, axs = plt.subplots(1, 3, figsize=(11, 3))
    axs[0].plot(x, get_rho(a, core_params))
    axs[0].set_title(r'$\rho$')
    axs[0].set_ylim([mins[0], maxs[0]])
    
    axs[1].plot(x, get_u(a, core_params))
    axs[1].set_title(r'$u$')
    axs[1].set_ylim([mins[1], maxs[1]])
    
    axs[2].plot(x, get_p(a, core_params))
    axs[2].set_title(r'$p$')
    axs[2].set_ylim([mins[2], maxs[2]])

def plot_trajectory(trajectory, core_params, mins = [0.0 - 2e-2] * 3, maxs= [1.0 + 5e-2] * 3):
    nx = trajectory.shape[2]
    xs = jnp.arange(nx) * core_params.Lx / nx
    xs = xs.T.reshape(-1)
    coords = {
        'x': xs,
        'time': t_inner * jnp.arange(trajectory.shape[0])
    }
    rhos = trajectory[:, 0, :]
    xarray.DataArray(rhos, dims=["time", "x"], coords=coords).plot(
        col='time', col_wrap=5)

    us = trajectory[:, 1, :] / trajectory[:, 0, :]
    xarray.DataArray(us, dims=["time", "x"], coords=coords).plot(
        col='time', col_wrap=5)

    ps = (core_params.gamma - 1) * (trajectory[:, 2, :] - 0.5 * trajectory[:, 1, :]**2 / trajectory[:, 0, :])
    xarray.DataArray(ps, dims=["time", "x"], coords=coords).plot(
        col='time', col_wrap=5)

In [ ]:
kwargs_core_params = {'Lx': 1.0, 'gamma': 1.4, 'bc': 'ghost', 'fluxstr': 'musclcharacteristic'}
kwargs_sim = {'name' : "test_euler", 'cfl_safety' : 0.2, 'rk' : 'ssp_rk3'}

core_params = get_core_params(**kwargs_core_params)
sim_params = get_sim_params(**kwargs_sim)
sim = EulerFVSim(core_params, sim_params)


nx = 400
f_init = shock_tube_problem_1(core_params)
a0 = get_a0(f_init, core_params, nx)

# simulate for some time
t_inner = 0.1
outer_steps = 10

inner_fn = get_inner_fn(sim.step_fn, sim.dt_fn, t_inner)
trajectory_fn = get_trajectory_fn(inner_fn, outer_steps)
trajectory = trajectory_fn(a0)

In [ ]:
maxs = [1.1, 1.5, 1.1]
plot_trajectory(trajectory, core_params, maxs=maxs)

### Plot entropy over time

The entropy for the Euler equations is $$s=\log(p/\rho^\gamma).$$

Consider the generalized entropy function $$h(s) = \rho g(s).$$ So long as $$\frac{d^2 g}{ds^2} / \frac{dg}{ds} < \frac{1}{\gamma}$$ then $h$ satisfies an entropy conservation law. Some otions for $g(s)$ include $g(s) = s$ as well as $g(s) = e^{\frac{s}{\gamma + 1}}$.

In [ ]:
vmap_get_entropy = vmap(lambda a: get_entropy(a, core_params))

N_ave = 1

Nt, _, nx = trajectory.shape
assert nx % N_ave == 0
downsampled_trajectory = jnp.mean(trajectory.reshape(Nt, 3, nx//N_ave, N_ave), axis=-1)

dx = core_params.Lx / downsampled_trajectory.shape[2]

entropy = vmap_get_entropy(downsampled_trajectory) * dx
total_entropy = jnp.sum(entropy, axis=1) 

In [ ]:
plt.plot(total_entropy)

In [ ]:
plt.plot((jnp.sum(trajectory[:,2],axis=-1) * dx))
print(jnp.sum(trajectory[:,2],axis=-1) * dx)

In [ ]:
plt.plot(jnp.sum(trajectory[:,2,1:-1],axis=-1) * dx)

In [ ]:
plt.plot(jnp.sum(trajectory[:,0,1:-1],axis=-1) * dx)

In [ ]:
print(trajectory.shape)
print(jnp.sum(trajectory[:,0,:],axis=-1) * dx)
print(jnp.sum(trajectory[:,1,:],axis=-1) * dx)
print(jnp.sum(trajectory[:,2,:],axis=-1) * dx)